In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [2]:
pd.set_option('display.max_colwidth', 1000)

Данные: датасет НГ  https://github.com/mannefedov/ru_kw_eval_datasets

In [3]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

Возьмём функцию оценивания из опорного ноутбука

In [4]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    #print('Precision - ', round(np.mean(precisions), 2))
    #print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    #print('Jaccard - ', round(np.mean(jaccards), 2))

Посмотрим, как выглядят наборы ключевых слов

In [7]:
data.keywords.head(10)

0                                  [школа, образовательные стандарты, литература, история, фгос]
1                                                                              [красота, законы]
2                                                [юзефович, гражданская война, пепеляев, якутия]
3                                                    [формула1, автоспорт, гонки, испания, квят]
4    [есенин, православие, святая русь, поэзия, год литературы, клюев, мариенгоф, стихи, россия]
5                             [медвузы, медицинское образование, рудн, николай стуров, интервью]
6                  [литература, книги, периодика, космос, небо, астрономия, анатомия, филология]
7                                                                             [сша, ирак, война]
8                                        [искусственный интеллект, робот, компьютер, технологии]
9                                                                  [вб, вто, переговоры, тарифы]
Name: keywords, dtype: object

### Наблюдения:
1. Помимо существительных, в ключевых словах встречаются прилагательные
2. Среднее количество слов, вероятно, меньше 10
3. Сохранено мн.ч. существительного

Проверим 2-е утверждение

In [17]:
s = 0
for i in data.keywords:
    s += len(i)
print(s/len(data.keywords))

6.181174089068826


И посмотрим, что происходит в нормализации с числом

In [20]:
morph = MorphAnalyzer()
morph.parse('законы')

[Parse(word='законы', tag=OpencorporaTag('NOUN,inan,masc plur,accs'), normal_form='закон', score=0.714285, methods_stack=((<DictionaryAnalyzer>, 'законы', 33, 9),)),
 Parse(word='законы', tag=OpencorporaTag('NOUN,inan,masc plur,nomn'), normal_form='закон', score=0.285714, methods_stack=((<DictionaryAnalyzer>, 'законы', 33, 6),))]

Воспользуемся "лучшими" способами оценки из опорного ноутбука, с учётом трёх наблюдений.  
Будем применять их по отдельности и в комбинациях.

а) изменим топ частотных слов

In [32]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [33]:
data['content_norm'] = data['content'].apply(normalize)

In [36]:
# топ 6 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(6)]))

F1 -  0.17


** Выбор топ-6 вместо топ-10 частотных слов статьи привел к F1 > бейзлайна**

b. Изменим только POS

In [38]:
def normalize_pos(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS in ['NOUN', 'ADJF']]

    return words

In [42]:
data['content_norm_pos'] = data['content'].apply(normalize_pos)

In [50]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm_pos'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

F1 -  0.15


*Дополнительный выбор прилагательных ухудшил результат по сравнению с бейзлайном*

c. Добавим прилагательные и выберем топ-6

In [61]:
# топ 6 частотных слов статьи
evaluate(data['keywords'], data['content_norm_pos'].apply(lambda x: [x[0] for x in Counter(x).most_common(6)]))

F1 -  0.15


*сочетание с выбором топ-6 не помогло*

d. Приведём слово не в нормальную форму, а в И.п.

In [46]:
def normalize_form(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.inflect({'nomn'}).word for word in words if word.tag.POS == 'NOUN']

    return words

In [69]:
data['content_norm_form'] = data['content'].apply(normalize_form)

In [54]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm_form'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

F1 -  0.19


** Приведение слов к именительному падежу с сохранением числа привело к F1 > бейзлайна**

e. То же самое, но топ-6

In [63]:
# топ 6 частотных слов статьи
evaluate(data['keywords'], data['content_norm_form'].apply(lambda x: [x[0] for x in Counter(x).most_common(6)]))

F1 -  0.2


** Приведение слов к именительному падежу с сохранением числа и выбором топ-6 вместо топ-10 привело к F1 > бейзлайна**

f. Добавим прилагательные и будем приводить к и.п.

In [65]:
def normalize_form_pos(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.inflect({'nomn'}).word for word in words if word.tag.POS in ['NOUN', 'ADJF']]

    return words

In [75]:
data['content_norm_form_pos'] = data['content'].apply(normalize_form_pos)

In [76]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm_form_pos'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

F1 -  0.18


In [77]:
# топ 6 частотных слов статьи
evaluate(data['keywords'], data['content_norm_form_pos'].apply(lambda x: [x[0] for x in Counter(x).most_common(6)]))

F1 -  0.19


** Добавление прилагательных ухудшает результат :(**

### То же с tfidf

In [81]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [82]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])

In [83]:
# сортируем и возьмём топ-6
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]] 

In [84]:
evaluate(data['keywords'], keywords)

F1 -  0.17


** Вот и отлично, снова помогло**

In [202]:
data['content_norm_pos_str'] = data['content_norm_pos'].apply(' '.join)
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_pos_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_pos_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 
evaluate(data['keywords'], keywords)

F1 -  0.15


In [203]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]] 
evaluate(data['keywords'], keywords)

F1 -  0.16


* Прилагательные делают хуже *

Попробуем с приведением к и.п.

In [204]:
data['content_norm_form_str'] = data['content_norm_form'].apply(' '.join)
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_form_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_form_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 
evaluate(data['keywords'], keywords)

F1 -  0.17


In [205]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]] 
evaluate(data['keywords'], keywords)

F1 -  0.19


** Стало лучше! **

In [206]:
data['content_norm_form_pos_str'] = data['content_norm_form_pos'].apply(' '.join)
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)
tfidf.fit(data['content_norm_form_pos_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_form_pos_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]] 
evaluate(data['keywords'], keywords)

F1 -  0.18


*и вновь прилагательные не помогают*

In [131]:
keywords[:5]

[['ольга васильев', 'стандарты', 'васильев', 'ольга', 'источник', 'письмо'],
 ['красота', 'отчаяние', 'кошка', 'глаза', 'скамейка', 'познание'],
 ['якутия', 'книга', 'восстание', 'сибирь', 'леонид', 'места'],
 ['гонка', 'команда', 'пилоты', 'сезон', 'при', 'гран при'],
 ['есенин', 'поэт', 'борода', 'смерть', 'стихи', 'лев']]

возможно, биграммы-имена собственные вроде "ольга васильев" немного засоряют нам результаты

### попробуем tfidf и топ-6 без биграмм

In [208]:
tfidf = TfidfVectorizer(min_df=5)
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]] 
evaluate(data['keywords'], keywords)

F1 -  0.18


**Без учёта биграмм стало лучше**

Приведение к И.п. без биграмм:

In [209]:
data['content_norm_form_str'] = data['content_norm_form'].apply(' '.join)
tfidf = TfidfVectorizer(min_df=5)
tfidf.fit(data['content_norm_form_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_form_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]] 
evaluate(data['keywords'], keywords)

F1 -  0.19


*Так же, как и с биграммами*

In [210]:
data['content_norm_form_pos_str'] = data['content_norm_form_pos'].apply(' '.join)
tfidf = TfidfVectorizer(min_df=5)
tfidf.fit(data['content_norm_form_pos_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors = tfidf.transform(data['content_norm_form_pos_str'])
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]] 
evaluate(data['keywords'], keywords)

F1 -  0.18


*Ещё один повод убедиться, что прилагательные не помогают*

### Итог:
- сокращение топа частотных слов с 10 до 6 приводит к превышению бейзлайна во всех трёх случаях
- приводить слово не к нормальной форме, а к Им.п. приводит к превышению бейзлайна во всех трёх случаях
- добавление прилагательных не делает результат лучше, а иногда ухудшает
- применение tfidf в сочетании с описанными приёмами позволяет побить бейзлайн, но не приводит к улучшению F1 по сранению с использованием топа частотных слов
- рассмотрение только униграмм улучшает результат только для топ-6 без изменения в нормализации

- лучший результат: F1 = 0.2 получен при выборе 6 самых частотных слов из статьи с приведением слов к форме Им.п. вместо нормальной формы.

### RAKE
Попробуем готовую имплементацию

In [132]:
from rake_nltk import Metric, Rake

In [176]:
r = Rake(language='russian', stopwords=stopwords.words('russian'))

In [177]:
type(data['content'])

pandas.core.series.Series

In [178]:
cont = pd.Series.tolist(data['content'])

In [179]:
r.extract_keywords_from_text(cont[0])

In [181]:
r.get_ranked_phrases()[:10]

['обозревателем « нг » ольга васильева прокомментировала претензии гильдии словесников',
 'каждому предмету разработано 15 – 20 конкретных требований',
 'министерство образования дает право контролирующим органам ловить детей',
 '« сформированность умений проводить атрибуцию текстового исторического источника',
 'прежних стандартах результатом изучения курса истории должно',
 'каждого года обучения приведены предметные результаты обучения',
 '– заявила « нг » ольга васильева',
 'накануне гильдия словесников разместила открытое письмо',
 'новые фгосы грубо нарушают права детей',
 'федеральных государственных образовательных стандартах начального общего']

Кажется, можно даже не считать F1  
Что если уменьшить длину фраз?

In [190]:
r = Rake(language='russian', stopwords=stopwords.words('russian'), min_length=1, max_length=2)

In [191]:
cont = pd.Series.tolist(data['content'])

посмотрим на одном тексте

In [192]:
r.extract_keywords_from_text(cont[0])

In [193]:
r.get_ranked_phrases()[:10]

['число книг',
 'цели создания',
 'участников событий',
 'увязываю это',
 'точки зрения',
 'существующему стандарту',
 'станет понятно',
 'средней школы',
 'сохранена структура',
 'сонме романов']

не обнадёживает

In [186]:
r = Rake(language='russian', stopwords=stopwords.words('russian'), min_length=1, max_length=1)

In [187]:
cont = pd.Series.tolist(data['content'])

In [188]:
r.extract_keywords_from_text(cont[0])

In [189]:
r.get_ranked_phrases()[:10]

['–',
 'явлениях',
 'явления',
 'явлений',
 'читают',
 'читали',
 'читала',
 'числе',
 'фгосам',
 'фгос']

### Итог
Rake "из коробки" не подходит. Надо что-то делать